# Regolith formation: Your turn!

In [ ]:
import numpy as np
import xsimlab as xs
import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors
import xarray as xr
import dask
import scipy.signal as sg
import zarr

# plt.style.use("dark_background")
%load_ext xsimlab.ipython

In [ ]:
from duricrust import regolith_model

In [ ]:
model = regolith_model

In [ ]:
model.visualize(show_inputs=True)

## Uplift/Precipitation exercise

Learn how to use the model. We will change values in uplift and precipitation, and analyse them. Goal: Look at constant U and P values and compare with periodic U and P values. For this, we will use the periodic square function.

#### Step 1: Analaysis of the regolith model

#### Step 2: What are the necessary parameters to run the model?

In [ ]:
import scipy.signal as sg



#### Step 3: fill in the blanks!

In [ ]:
%create_setup model -d -v

#### Step 4: How many values are in the uplift rate array?

#### Step 5: run the model + store it

In [ ]:
zgroup = zarr.group("regolith.zarr", overwrite=True)

with model, xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(store=zgroup)
    
ds_out

#### Step 5: Parameter check: What can we analyse?

#### Step 6: PLOTS, lots of (nice) plots

##### Plot 1: Uplift and precipitation through time, in 2 boxes.

##### Plot 2: Compare uplift with topography

##### Plot 3: On 1 plot, mean topography, mean weatherring front height and mean regolith thickness through time

##### Plot 4: Plot a profile, with topography, regolith thickness and water table position at the last time step.

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from tqdm.auto import tqdm

out=-1
fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(9,3))

# The actual plotting


#Fluffing
ax.text(430, ds_out.regolith__water_table.isel(out=out).isel(x=10)-10, 'Water Table', color="red")
ax.arrow(400, ds_out.regolith__water_table.isel(out=out).isel(x=10),35,-6, width = 1, length_includes_head = True)

nx2 = int(len(ds_out.x)/2)
ax.text(ds_out.x[nx2]+55, ds_out.topography__elevation.isel(out=out)[nx2]+4, 'Surface Topography', color="darkgreen")
ax.arrow(ds_out.x[nx2]+55, ds_out.topography__elevation.isel(out=out)[nx2]+5, -10, -5,
                    width = 1, length_includes_head = True)
ax.text(ds_out.x[-20]-510, ds_out.regolith__weathering_front.isel(out=out)[-20]-29, 'Weathering Front', color="darkorange")
ax.arrow(ds_out.x[-10]-510, ds_out.regolith__weathering_front.isel(out=out)[-40]-10, -10, -6, width = 1, length_includes_head = True)
    
ax.text(ds_out.x[-20], ymin+23, f'Time = {(ds_out.out[-1]/1e6).values:.2f} Myr')

#Axis definition and space allocation


##### Plot 5: Same, but at different time steps. Make it 6.

##### Plot 6: A video of the profile through time, with time link.

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from tqdm.auto import tqdm
#The movie

fig,ax = plt.subplots(nrows=2, ncols=1, figsize=(10,6), gridspec_kw={'height_ratios': [3, 1]})

ymin = ds_out.regolith__weathering_front.min()
ymax = ds_out.topography__elevation.max()

for i, out in tqdm(enumerate(ds_out.out[:])):
    x,y = np.meshgrid(ds_out.x,ds_out.y)
    for k in range(int(ds_out.mesh__nx)):
        y[:,k] = (
            y[:,k]*ds_out.regolith__thickness.sel(out=out).values[k]
            + ds_out.regolith__weathering_front.sel(out=out).values[k]
        )
    ax[0].clear()
    ax[0].set_ylim(ymin, ymax)

#Plotting
    

#Fluffing up the graphs    
    
    ax[0].text(10, ds_out.regolith__water_table.sel(out=out)[0]+20, 'Base Level')
    ax[0].arrow(10, ds_out.regolith__water_table.sel(out=out)[0]+17, -10, -17, width = 1, length_includes_head = True)
    nx2 = int(len(ds_out.x)/2)
    if i<150:
        ax[0].text(ds_out.x[nx2]+10, ds_out.topography__elevation.sel(out=out)[nx2]+20, 'Surface Topography')
        ax[0].arrow(ds_out.x[nx2]+10, ds_out.topography__elevation.sel(out=out)[nx2]+17, -10, -17,
                    width = 1, length_includes_head = True)
    ax[0].text(ds_out.x[10]+10, ds_out.regolith__water_table.sel(out=out)[10]-27, 'Water Table')
    ax[0].arrow(ds_out.x[10]+10, ds_out.regolith__water_table.sel(out=out)[10]-17, -10, 17, width = 1, length_includes_head = True)
    ax[0].text(ds_out.x[-20]+10, ds_out.regolith__weathering_front.sel(out=out)[-20]-27, 'Weathering Front')
    ax[0].arrow(ds_out.x[-20]+10, ds_out.regolith__weathering_front.sel(out=out)[-20]-17, -10, 17, width = 1, length_includes_head = True)
    
    ax[0].text(ds_out.x[-25], ymin+10, f'Time = {(ds_out.out[i]/1e6).values:.2f} Myr')
    
    ax[0].xaxis.tick_top()
    ax[0].xaxis.set_label_position('top') 
    ax[0].set_xlabel('Distance (m)')
    ax[0].set_ylabel('Height (m)')

    ax[0].text(ds_out.x[-50], ymin+30, 'Basement')
    if i>100: ax[0].text(ds_out.x[-65], ymin+90, 'Regolith')
    
    ax[1].clear()
    ax[1].plot(ds_out.out[:i]/1e6, ds_out.uplift__rate.sel(time=ds_out.out)[:i]*1e6)
    ax[1].text(ds_out.out[i]/1e6,ds_out.uplift__rate.sel(time=ds_out.out)[i]*1e6, 'x', ha='center', va='center')
    
    ax[1].set_xlim(0, ds_out.out[-1]/1e6)
    ax[1].set_ylim(-ds_out.uplift__rate.max()*0.1e6, ds_out.uplift__rate.max()*2.1e6)
    ax[1].set_xlabel('Time (Myr)')
    ax[1].set_ylabel('Uplift Rate (m/Myr)')

    fig.savefig('Frames/Frame'+str(i)+'.png', dpi=150) #dpi is the resolution! nb of frames is in my linspace 0 tfinal 101!


##### Plot 7: Omega and Gamma through time

What does it mean? What can you see through these values? Is it what we observe?

In [ ]:
fig,ax = plt.subplots(nrows=2, ncols=1, figsize=(10,6))

Comparing Gamma to the Gamma/Omega ratio will give you some part of the answer. We can create this comparison by calculating and adding attributes to a plot line:

When Gamma is smaller than the ratio: rego thickest at the base.

##### Plot 8: Weathering age

If we want to check out the next parts, we will need another model including duricrusts. However, we will determine parameters so that no duricrust will form.

In [ ]:
from duricrust import water_table_model
WTmodel = water_table_model

In [ ]:
import scipy.signal as sg

tfinal = 10e6 #time to the end
nstep = 10001 #time step
tim = np.linspace(0, tfinal, nstep)
U0 = 50e-6
nhumps = 10
U = U0 * (1 + sg.square(np.linspace(0, 2 * np.pi * nhumps, nstep), duty=0.5)) / 2 
U = xr.DataArray(U, dims="time") #to specify the dimension we want to use, so he can work with it in the right dimensions.
U = U/U.mean()*U0
print('mean uplift rate', U.mean().values)

P0 = 5
P = np.ones(nstep)
P = xr.DataArray(P, dims="time")
P = P/P.mean()*P0
print('mean precipitation rate', P.mean().values)

In [ ]:
# %create_setup WTmodel -d -v
import xsimlab as xs

ds_in = xs.create_setup(
    model=WTmodel,
    clocks={"time": np.linspace(0, tfinal, nstep),
            "out": np.linspace(0, tfinal, 201)},
    master_clock = "time",
    input_vars={
        # number of nodes in the horizontal x-direction
        'mesh__nx': 101,
        # horizontal length of the model
        'mesh__L': 1000,
        # number of points in the vertical y-direction
        'mesh__ny': 501,
        # precipitation rate
        'precip__rate': P,
        # uplift rate
        'uplift__rate': U,
        # hydraulic conductivity
        'regolith__K': 1e4,
        # ratio of weathering front velocity over fluid velocity
        'regolith__F': 1e-6,
        # number of points in the vertical y-direction
        'duricrust__ny': 501,
        # characteristic time scale for duricrust formation
        'duricrust__tau': 10e30,
        # water table beating height
        'duricrust__lamda': 0.1,
        # reference diffusivity
        'harden_diffusion__Kd': 1,
        # initial topographic slope
        'init_topography__slope': 0.01,
    },
    output_vars={"topography__elevation": "out",
        "regolith__thickness": "out",
        "regolith__water_table": "out",
        "regolith__weathering_front": "out",
        "duricrust__hardness": "out",
        "duricrust__age_regolith": "out",
        "duricrust__age_duricrust": "out",
        "regolith__Omega": "out",
        "regolith__Gamma": "out",}
)


In [ ]:
zgroup = zarr.group("regolith_duri.zarr", overwrite=True)

with WTmodel, xs.monitoring.ProgressBar():
    ds_out = ds_in.xsimlab.run(store=zgroup)
    
ds_out

In [ ]:
out=-1
fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(9,4)) #2 diagrams in 1 plot

#The diagram colour base. Without it, you won't see colors. This step is needed as it defines the characteristics of the diagram mesh
x, y = np.meshgrid(ds_out.x, ds_out.y)
for k in range(int(ds_out.mesh__nx)):
    y[:, k] = (
        y[:, k] * ds_out.regolith__thickness.isel(out=out).values[k] + 
        ds_out.regolith__weathering_front.isel(out=out).values[k]
    )

# The actual plotting
#cp = ax.contourf(x, y, (ds_out.out.isel(out=-1) - ds_out.duricrust__age_regolith.isel(out=out)), np.linspace(0,2*ds_out.out[-1].values/nhumps,11))

##### Plot 9: Fluxes in the regolith

Vizualisation of sediment fluxes. For this, we need to adjust the time scale, for it to work in time and not in model time. Uplift will be changed into an array filled with 1, and then our uplift values (all?) will become a cumulative sum with the attribut ".cumsum()'.

In [ ]:
uplift = xr.ones_like(ds_out.time) #"uplift" is now a array with only ones as values. The array is as long as ds_out.time to match it correctly.
uplift[1:] = (ds_out.uplift__rate[1:]*ds_out.time.diff('time')).cumsum() #here, we say that uplift[1:] is the value of the cumulative sum of the calculation.

Now we get to define our fluxes according to erosion, weathering front "moving" rate and everything will have to be scaled.

In [ ]:
#Plotting 4 parameters in 1 diagram. Axis x (out) is time, axis y is all the other parameters.

fig, ax = plt.subplots(figsize=(12,5))

#Definition of values we want to plot
erosion_rate = uplift.sel(time=ds_out.out).differentiate('out') - ds_out.topography__elevation.differentiate('out')
front_rate = uplift.sel(time=ds_out.out).differentiate('out') - ds_out.regolith__weathering_front.differentiate('out')

sediment_flux = erosion_rate.integrate('x')
sediment_flux.plot(ax=ax, label='Sediment Flux')

weathering_flux = front_rate.integrate('x')
weathering_flux.plot(ax=ax, label='Weathering Flux')

#The actual plotting of uplift and precipitation
(ds_out.uplift__rate.sel(time=ds_out.out)*ds_out.mesh__L).plot(ax=ax, label='uplift function')
scale = ds_out.uplift__rate.sel(time=ds_out.out).max()*ds_out.mesh__L/ds_out.precip__rate.sel(time=ds_out.out).max()
(ds_out.precip__rate.sel(time=ds_out.out)*scale).plot(ax=ax, label='precip function')

ax.legend()

##### Plot 10: Residence time

It is possible to compute the residence time of praticles in the regolith, when we know the sediment and weathering fluxes. It must be done in two steps:
- First, we define what are our residence times in the regolith and in the water.
- Then we can plot
Step 1:

In [ ]:
#definition and calculation for the residence time, plotted in the cell below

residence_time_regolith = xr.zeros_like(ds_out.out)
residence_time_water = xr.zeros_like(ds_out.out)

for i,t in enumerate(ds_out.out):
    func = ((uplift.sel(time=ds_out.out).sel(out=t)-uplift.sel(time=ds_out.out)
      -1*(ds_out.topography__elevation.sel(out=t)-ds_out.topography__elevation)
     -ds_out.regolith__thickness).mean('x'))
    zero_crossings = np.where(np.diff(np.sign(func)))[0]
    tb = 0
    if len(zero_crossings)>0:
        tb = (ds_out.out[1+zero_crossings[0]]
              + (ds_out.out[2+zero_crossings[0]] - ds_out.out[1+zero_crossings[0]])
             * (0 - func[zero_crossings[0]])
              / (func[zero_crossings[0]+1] - func[zero_crossings[0]]))
        residence_time_regolith[i] = ((t-tb).values)
    funcw = ((uplift.sel(time=ds_out.out).sel(out=t)-uplift.sel(time=ds_out.out)
      -1*(ds_out.topography__elevation.sel(out=t)-ds_out.topography__elevation)
     -(ds_out.topography__elevation-ds_out.regolith__water_table)).mean('x'))
    zero_crossings = np.where(np.diff(np.sign(funcw)))[0]
    if len(zero_crossings)>0:
        tw = (ds_out.out[1+zero_crossings[-1]]
              + (ds_out.out[2+zero_crossings[-1]] - ds_out.out[1+zero_crossings[-1]])
             * (0 - funcw[zero_crossings[-1]])
              / (funcw[zero_crossings[-1]+1] - funcw[zero_crossings[-1]])) 
        residence_time_water[i] = (tw- tb).values

Step 2:

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

#Plotting of the residence time compared to other parameters
residence_time_regolith.plot(ax=ax, label='residence time in regolith')

residence_time_water.plot(ax=ax, label='residence time below water table in regolith')

water_flux = ds_out.precip__rate.sel(time=ds_out.out)*residence_time_water
(water_flux/ds_out.precip__rate.max()/2).plot(ax=ax, label='water flux')

scale = residence_time_regolith.max()/ds_out.uplift__rate.max()
(ds_out.uplift__rate.sel(time=ds_out.out)*scale).plot(ax=ax, label='uplift function')

scale = residence_time_regolith.max()/ds_out.precip__rate.max()
(ds_out.precip__rate.sel(time=ds_out.out)*scale).plot(ax=ax, label='precip function')

ax.legend()
fig.tight_layout()

##### Plot 11: Solubility

Variation in solubility, variation in F, can be expressed as dependent on temperature through the Arrhenius law which gives us this diagram following the Arhenius equation.

In [ ]:
plt.plot(np.exp(-1107/(274+np.linspace(0,50))-0.0254))

## Batch uplift: analysis of multiple U at the same time, anorogenic to orogenic environments.

In [ ]:
import scipy.signal as sg

tfinal = 10e6 #time to the end
nstep = 10001 #time step
tim = np.linspace(0, tfinal, nstep)
U0 = 50e-6
# choice of a step-wise uplift rate with nhumps number of steps
nhumps = 10
U = U0 * (1 + sg.square(np.linspace(0, 2 * np.pi * nhumps, nstep), duty=0.5)) / 2 
#The 1/5 duty makes the percent of the signal period in which the square wave is positive,
# which means that by 1/2, 50% of the wave is positive and 50 negative
# choice of a constant uplift rate
#U = U0*np.ones(nstep)
U = xr.DataArray(U, dims="time") #to specify the dimension we want to use, so he can work with it in the right dimensions.
U = U/U.mean()*U0
print('mean uplift rate', U.mean().values)

P0 = 5
#choice of a step-wise precipitaiton function
#P = 5 + 10 * sg.square(np.linspace(0, 2 * np.pi * 10, nstep), duty=0.5) / 2
# choice of a uniform precipitation function
P = np.ones(nstep)
P = xr.DataArray(P, dims="time")
P = P/P.mean()*P0
print('mean precipitation rate', P.mean().values)

In [ ]:
%create_setup model -d -v

Question: What happens in anorogenic areas? And what happens in orogenic areas? What's the difference?

## Batch precipitation: analysis of multiple P at the same time.

In [ ]:
import scipy.signal as sg

tfinal = 10e6 #time to the end
nstep = 10001 #time step
tim = np.linspace(0, tfinal, nstep)
U0 = 50e-6
# choice of a step-wise uplift rate with nhumps number of steps
nhumps = 10
U = U0 * (1 + sg.square(np.linspace(0, 2 * np.pi * nhumps, nstep), duty=0.5)) / 2 
#The 1/5 duty makes the percent of the signal period in which the square wave is positive,
# which means that by 1/2, 50% of the wave is positive and 50 negative
# choice of a constant uplift rate
#U = U0*np.ones(nstep)
U = xr.DataArray(U, dims="time") #to specify the dimension we want to use, so he can work with it in the right dimensions.
U = U/U.mean()*U0
print('mean uplift rate', U.mean().values)

P0 = 5
#choice of a step-wise precipitaiton function
#P = 5 + 10 * sg.square(np.linspace(0, 2 * np.pi * 10, nstep), duty=0.5) / 2
# choice of a uniform precipitation function
P = np.ones(nstep)
P = xr.DataArray(P, dims="time")
P = P/P.mean()*P0
print('mean precipitation rate', P.mean().values)

In [ ]:
%create_setup model -d -v